# PyTorch神经网络
在pytorch中，神经网络使用torch.nn来构建。神经网络是基于自动梯度来定义的一些模型，一个nn.Moudle包括层和一个方法forward()，它会返回输出。  

比如：看下面数字图片识别网络:

![figure.1](https://gitee.com/zyp521/upload_image/raw/master/oDyZLc.png)

这是一个简单的前馈神经网络，它接收输入，让输出一个接着一个的通过一些层，最后给出输出。  

一个经典的神经网络训练过程包含下列几点:
- (1)定义一个包含可训练参数的神经网络
- (2)迭代整个输入
- (3)通过神经网络处理输入
- (4)计算损失loss
- (5)反向传播梯度到神经网络的参数
- (6)更新网络的参数，比较经典的参数更新方法为:
```
weight = weight - learning_rate * gradient # learning_rate表示对梯度的接受程度
```

## 模型定义
下面来定义一个神经网络:

In [51]:
import torch
import torch.nn as nn
import torch.nn
import torch.nn.functional as F

class Net(nn.Module):
    # fixme: 网络搭建
    def __init__(self):
        super(Net, self).__init__()
        # build model
        self.conv1 = nn.Conv2d(1, 6, 5) # input channel为1 output channel 为 6, 5×5 kernal的 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5) # I为6 O为16 kernal为5×5
        # fc layer
        self.fc1 = nn.Linear(16*5*5, 120) # Input为400 Ouput为120
        self.fc2 = nn.Linear(120, 84) # Input为120 Ouput为84
        self.fc3 = nn.Linear(84, 10) # Input为84 Output为10
    
    # fixme: 扁平化
    def num_flat_features(self, x):
        size = x.size()[1:] # batch数量
        num_features = 1
        # 按照batch来划分
        for s in size:
            num_features *= s
        return num_features
    
    # fixme: 损失函数
    def loss(self, output, target):
        criterion =  nn.MSELoss() # 调用MESLoss计算loss
        return criterion(output, target)

    # fixme: 前向传播
    def forward(self, x):
        """
        :param x: 输入图片
        """
        # 卷积层1
        x = self.conv1(x)
        # activation function
        x = F.relu(x)
        # max pooling 2×2 window
        x = F.max_pool2d(x,(2,2))
        # 卷积层2
        x = self.conv2(x)
        # activation
        x = F.relu(x)
        # max pooling 2×2
        x = F.max_pool2d(x,2) # 这里输入2，如果是2×2的window的话，则等价于(2,2)
        # flatten
        x = x.view(-1, self.num_flat_features(x)) # 将特征拉成一条
        # Fc Layer
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


这里定义了一个前馈函数，然后反向传播函数被自动通过autograd定义了。你可以使用任何张量操作在前馈函数上。  
一个模型可训练的参数可以通过调用net.parameters()返回:

In [33]:
params = list(net.parameters())
print(len(params))
#print(params)
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


这里尝试随机生成32×32的输入，输入的维度是32×32. 为了能够让数据集在该网络上运行，需要将图片的维度修改为32×32.

## 正向传播

In [44]:
input = torch.randn(4, 1, 32, 32) # 生成维度为1×32×32 batch中的元素个数为4
input1 = torch.randn(1, 1, 32, 32) # 生成维度为1×32×32 batch中的元素个数为1
# print(input)
output = net(input)
print(output)

tensor([[ 0.0466,  0.1115, -0.0826,  0.1214,  0.0293, -0.0469, -0.1374, -0.2011,
          0.0472,  0.0341],
        [ 0.0588,  0.1029, -0.0577,  0.1265,  0.0065, -0.0465, -0.1586, -0.2081,
          0.0639,  0.0531],
        [ 0.0581,  0.1173, -0.0858,  0.1199,  0.0103, -0.0534, -0.1621, -0.2184,
          0.0457,  0.0284],
        [ 0.0475,  0.1148, -0.0773,  0.1166,  0.0198, -0.0303, -0.1578, -0.2353,
          0.0562,  0.0482]], grad_fn=<AddmmBackward>)


针对上面网络中一些函数的分析:
- (1) nn.Conv2d(x, y, z) 卷积层：构建一个接收通道数为x，输出通道数为y，kernel大小为 z×z
- (2) nn.Linear(m, n) 线性变换：输入维度为m，输出的维度为n
- (3) F.max_pool2d(x,m) max pooling：输入tensor为x，池化window的大小为m×m
- (4) F.max_pool2d(x,(m,n)) 和上面同理，这里(m,n) 当m=n时，(m,m) 等价于 m
- (5) F.relu(x) 激活函数
- (6) tensor.view(m, n) 将tensor重新变形，但是m×n 要和tensor原来的维度相同
- (7) params = list(net.parameters())得到每层对应的参数列表，params[0]对应一层，卷积层，激活函数都会被包含，但是max_pooling没有被包含

对于输入
```
input1 = torch.randn(1, 1, 32, 32)
```
的分析，这里因为网络要求的输入channel为1，因此输入的维度为1×32×32，对于前面的1这里指的是一个batch的大小。  

把所有的参数梯度缓存器置零，用随机的梯度来反向传播:

In [45]:
net.zero_grad() # 网络所有参数梯度置零
output.backward(torch.randn(4, 10)) # 4指的是batch的大小

上述工作做完之后，完成了下述任务：
- (1)定义了一个神经网络
- (2)正向传播以及调用反向传播

还需要完成:
- (1)计算loss
- (2)更新网络参数权重

## 反向传播
### 计算loss
计算loss代码:

In [64]:
output = net(input)
print(output.size())
target = torch.randn(4,10) # 生成4个batch对应的target
for i in range(len(target)):
    temp = target[i].view(-1,1) # 针对每个batch对应扁平化
    res = net.loss(output[i], temp) # 计算loss
    print(res)

torch.Size([4, 10])
tensor(0.9781, grad_fn=<MseLossBackward>)
tensor(1.0270, grad_fn=<MseLossBackward>)
tensor(1.2662, grad_fn=<MseLossBackward>)
tensor(0.6900, grad_fn=<MseLossBackward>)


当我们调用loss.backward()，整个图都会微分，而且所有在图中的requires_grad=True的张量将会让它们的grad张量累计梯度。  

反向传播例子代码:

In [66]:
# 正向传播
output = net(input1)
print(output.size())
target = torch.randn(1,10)
temp = target.view(-1,1)
loss = net.loss(output, temp) # 计算loss

# 反向传播
net.zero_grad() # 将所有张量的梯度换层置零
print('反向传播之前的梯度:')
print(net.conv1.bias.grad)

loss.backward() # 反向传播

print('反向传播之后的梯度')
print(net.conv1.bias.grad)

torch.Size([1, 10])
反向传播之前的梯度:
None
反向传播之后的梯度
tensor([ 0.0003,  0.0011, -0.0007, -0.0008, -0.0002, -0.0008])


### 更新网络参数
这里使用一个比较简单的更新规则来进行参数的更新:
```
weight = weight - learning_rate * gradient
```

使用torch.optim实现了所有的方法。这里直接调用包来进行优化：

In [74]:
import torch.optim as optim
# 创建优化器
optimizer = optim.SGD(net.parameters(), lr=0.01) # 使用随机梯度下降
optimizer.zero_grad() # 梯度置零
# 前向传播
input1 = torch.randn(1, 1, 32, 32)
output = net(input1)
target = torch.randn(10)
# 计算loss
loss = net.loss(output,target)
# 反向传播
loss.backward()
optimizer.step() # 参数梯度更新